In [1]:
import requests
from bs4 import BeautifulSoup
import urllib
import os

def download_images(keyword, limit):
    # Create a directory to store the downloaded images
    if not os.path.exists(keyword):
        os.makedirs(keyword)

    # Format the keyword for the Google Images search URL
    query = urllib.parse.quote(keyword)

    # Set the number of images to download per request
    images_per_request = 20

    # Calculate the number of requests needed to download the desired number of images
    num_requests = (limit // images_per_request) + 1

    for i in range(num_requests):
        # Calculate the start index for the current request
        start_index = i * images_per_request

        # Construct the Google Images search URL
        url = f"https://www.google.com/search?q={query}&tbm=isch&start={start_index}"

        # Set headers to simulate a browser request
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
        }

        # Send an HTTP GET request to the Google Images URL
        response = requests.get(url, headers=headers)

        # Parse the HTML content of the response
        soup = BeautifulSoup(response.content, "html.parser")

        # Find all img tags in the HTML
        images = soup.find_all("img")

        # Iterate over the found images and download them
        for j, img in enumerate(images):
            if j >= limit:
                break

            try:
                # Get the source URL of the image
                src = img["src"]

                # Send an HTTP GET request to download the image
                img_response = requests.get(src, stream=True)

                # Set the file path for saving the image
                file_path = f"{keyword}/{keyword}_{start_index + j}.jpg"

                # Save the image to the file path
                with open(file_path, "wb") as file:
                    file.write(img_response.content)

                print(f"Downloaded image {start_index + j + 1}/{limit}")
            except Exception as e:
                print(f"Error downloading image {start_index + j + 1}: {str(e)}")

            # Delay for a short period between image downloads (to be polite to the server)
            time.sleep(0.1)

# Usage: Provide the keyword and the number of images to download
keyword = "a"
limit = 100
download_images(keyword, limit)


Error downloading image 1: Invalid URL '/images/branding/searchlogo/1x/googlelogo_desk_heirloom_color_150x55dp.gif': No scheme supplied. Perhaps you meant http:///images/branding/searchlogo/1x/googlelogo_desk_heirloom_color_150x55dp.gif?


NameError: name 'time' is not defined

In [1]:
%time
import requests
from bs4 import BeautifulSoup

def get_images(query):
  """
  Get the first 100 images from Google Images for a given query.

  Args:
    query: The query to search for.

  Returns:
    A list of image URLs.
  """

  # Make a request to Google Images.
  response = requests.get("https://www.google.com/search?q=" + query)

  # Parse the response as HTML.
  soup = BeautifulSoup(response.content, "html.parser")

  # Find the image results.
  image_results = soup.find_all("img", class_="rg_i")

  # Get the image URLs.
  image_urls = [image["src"] for image in image_results]

  # Return the first 100 image URLs.
  return image_urls[:100]

def download_images(image_urls):
  """
  Download the images from a list of image URLs.

  Args:
    image_urls: A list of image URLs.

  """

  # Create a directory to store the images.
  directory = ""
  if not os.path.exists(directory):
    os.mkdir(directory)

  # Download the images.
  for image_url in image_urls:
    image_name = image_url.split("/")[-1]
    image_path = os.path.join(directory, image_name)
    response = requests.get(image_url)
    with open(image_path, "wb") as f:
      f.write(response.content)

def main():
  # Get the input query.
  query = input("Enter a query: ")

  # Get the image URLs.
  image_urls = get_images(query)

  # Download the images.
  download_images(image_urls)

if __name__ == "__main__":
  main()


CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 3.1 µs


NameError: name 'os' is not defined

In [2]:
%time
import numpy as np
import cv2 as cv
import os
import faiss

CPU times: user 3 µs, sys: 3 µs, total: 6 µs
Wall time: 12.9 µs


In [3]:
def get_sift_keypoints(img):
    gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    sift = cv.SIFT_create(nfeatures=5000)
    kp, des = sift.detectAndCompute(gray, None)
    return kp, des

In [11]:
labels = []
features = []
for image_name in os.listdir('images'):
    building_name = image_name[0:image_name.find('.')]
    
    image_path = 'images/' + image_name
    img = cv.imread(image_path)
    
    kp, des = get_sift_keypoints(img)
    labels += [building_name for i in range(len(kp))]
    
    features.append(np.vstack(des))
    
    # img=cv.drawKeypoints(gray,kp,img)
    # cv.imshow('image',img)
    # cv.waitKey(0)

In [12]:
len(labels)

112722

In [13]:
features_ar = np.vstack(features)
features_ar.shape

(112722, 128)

In [14]:
index = faiss.IndexFlatL2(128)
index.add(features_ar)
print(index.ntotal)

112722


In [15]:
def find_closest_image_match(img):
    return None

In [16]:
search_image_path = 'search.jpg'
img = cv.imread(search_image_path)
kp, des = get_sift_keypoints(img)
k = 1

In [17]:
preds = {}
for d in des:
    D, I = index.search(d.reshape((1, 128)), k)
    pred = labels[I[0, 0]]
    if pred not in preds:
        preds[pred] = 0
    preds[pred] += 1

preds

{'Norris_University_Center_3': 8,
 'Walter_Annenberg_Hall_1': 10,
 'Fisk_Hall_4': 20,
 'Swift_Hall_2': 5,
 'The_Rock_1': 41,
 'Harris_Hall_2': 30,
 'The_Virginia_Wadsworth_Wirtz_Center_3': 83,
 'The_Rock_4': 16,
 'Kellogg_Global_Hub_4': 25,
 'Pick_Staiger_Concert_Hall_1': 93,
 'Bienen_School_of_Music_4': 45,
 'Donald_P_Jacobs_Center_2': 3,
 'Bienen_School_of_Music_2': 69,
 'The_Virginia_Wadsworth_Wirtz_Center_1': 80,
 'Garrett-Evangelical_Theological_Seminary_1': 35,
 'kellogg': 148,
 'Norris_University_Center_2': 110,
 'kellogg_full': 222,
 'Mudd_Library_3': 11,
 'The_Virginia_Wadsworth_Wirtz_Center_4': 30,
 'Bienen_School_of_Music_1': 68,
 'Mudd_Library_1': 10,
 'University_Hall_2': 10,
 'Pick_Staiger_Concert_Hall_2': 140,
 'Ford_Motor_Company_Engineering_Design_Center_2': 16,
 'Ford_Motor_Company_Engineering_Design_Center_1': 12,
 'Deering_Library_2': 7,
 'University_Hall_3': 11,
 'Kellogg_Global_Hub_3': 12,
 'Crowe_Hall_1': 10,
 'Segal_Visitors_Center_4': 24,
 'University_Library_4